In [4]:
# Enter your script here.

import numpy as np
import pandas as pd
import csv
import re
import calendar
import datetime
format_str = '%Y-%m-%d' # The format
import functools 
import operator
import simplejson as json

def check_columns(data):
    for columns in data:
      print(columns + ":  " +  str(len(pd.unique(data[columns]))) )
      print(pd.unique(data[columns]))
      print('++++++++++++++++++')

def splitQuery(column):
    listOfthings = "'" + column.astype(str).str.cat(sep="', '") + "'"
    return listOfthings

In [5]:
dados_brutos = pd.read_csv('dados_brutos_feb2020.csv')
dados_cidades = pd.read_csv('municipios.csv')
dados_estados = pd.read_csv('estados.csv')

dados_brutos

dados_geo = dados_cidades.merge(dados_estados,left_on='codigo_uf', right_on='codigo_uf',suffixes=('_cidade', '_estado'))

In [6]:
dados_geo['cidade_estado'] = dados_geo['nome_cidade'] + ' - ' + dados_geo['nome_estado']

In [7]:
dados_brutos.columns = dados_brutos.columns.str.replace(' ', '_')

regex = r"\_\(\w*|\W*\)"
regex_under = r"(^\_)"

dict_para_novos_nomes = {}

for items in dados_brutos.columns:
    ##caixa baixa
    item_caixa_baixa = items.lower()
    ##tirar parenteses bizarros e underscore comeco
    novo_item = re.sub(regex,'',item_caixa_baixa)
    sem_under = re.sub(regex_under,'',novo_item)
    ##tirar co2 com superscript
    dict_para_novos_nomes[items] = sem_under.replace('co²','co2')

#renomear colunas aqui
dados_brutos.rename(columns = dict_para_novos_nomes,inplace=True)


In [9]:
dados_brutos

,cidade,estado,região,sistema,operadora,tipologia_do_veículo,estações,bicicletas_elétricas,bicicletas,patinetes_elétricos,viagens_diárias,média_distância_percorrida_por_dia,usuários,mulheres,homens,15_até_29_anos,30_até_59_anos,acima_de_60,emissões_de_co2_evitadas,fonte_e_escopo_temporal
0,Manaus,Amazonas,Norte,Manôbike,Tembici,Bicicleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Salvador,Bahia,Nordeste,Bike Salvador,Tembici,Bicicleta,50.0,NaN,400.00,NaN,1284.0,8263.34,NaN,28.55,65.83,40.61,51.13,3.95,0.2100,NaN
2,Fortaleza,Ceará,Nordeste,Bicicleta Integrada,Serttel,Bicicleta,7.0,NaN,350.00,NaN,128.0,NaN,8436.0,35.40,64.60,33.59,62.81,3.59,NaN,NaN
3,Fortaleza,Ceará,Nordeste,Bicicletar,Serttel,Bicicleta,85.0,NaN,850.00,NaN,1317.0,"16594,20",57078.0,49.68,50.32,64.74,33.83,1.43,0.3700,NaN
4,Brasília,Distrito Federal,Centro-Oeste,Yellow,Grow,Bicicleta,NaN,NaN,384.25,NaN,1216.0,3907.61,971.0,NaN,NaN,39.52,58.80,1.68,0.1800,NaN
5,Brasília,Distrito Federal,Centro-Oeste,Mais Bike,Serttel,Bicicleta,50.0,NaN,500.00,NaN,830.0,"6003,67",36665.0,40.44,59.56,51.89,46.70,1.41,0.2600,NaN
6,Brasília,Distrito Federal,Centro-Oeste,Yellow,Grow,Patinete Eletrico,NaN,NaN,NaN,NaN,NaN,4320.31,NaN,NaN,NaN,35.55,63.00,1.45,0.2000,NaN
7,Serra,Espírito Santo,Sudeste,Bike Serra,Serttel,Bicicleta,20.0,NaN,200.00,NaN,177.0,"2147,60",4148.0,49.04,50.96,40.02,58.32,1.66,0.0800,NaN
8,Vila Velha,Espírito Santo,Sudeste,Bike VV,Tembici,Bicicleta,20.0,NaN,200.00,NaN,2059.0,10686.35,NaN,30.91,59.60,37.46,47.25,3.24,0.3900,NaN
9,Vila Velha,Espírito Santo,Sudeste,Yellow,Grow,Patinete Eletrico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
dados_brutos.replace(to_replace=r'\%', value='', regex=True, inplace=True)
dados_brutos.replace(to_replace = 'dockless', value = np.NaN, inplace=True)

lista_colunas_trocar = ['estações','mulheres','homens','15_até_29_anos','30_até_59_anos','acima_de_60']

dicionario_final = {}
dicionario_final['local'] = 'Brasil'
dicionario_final['sistema_total'] = []
dicionario_final['sistema_por_estado'] = []

dict_siglas = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

def get_key(val,siglas): 
    for key, value in siglas.items(): 
         if val == value: 
             return key 
        
    return "key doesn't exist"

### dados agregados de sistemas

dados_brutos['estado'] = dados_brutos['estado'].str.rstrip()

for estados in pd.unique(dados_brutos['estado']):
    print(estados)
    dict_estado = {}
    dict_estado['estado'] = estados
    frame_estado = dados_brutos[dados_brutos['estado'] == estados]
    dict_estado['sistemas'] = []
    for sis_por_estado in pd.unique(frame_estado['sistema']):
        sigla = get_key(estados.rstrip(),dict_siglas)
        dicionario_final['sistema_total'].append(sis_por_estado + ' – ' + sigla)
        dict_estado['sistemas'].append(sis_por_estado + ' – ' + sigla)
    dicionario_final['sistema_por_estado'].append(dict_estado)

dicionario_final['operadora'] = pd.unique(dados_brutos['operadora']).tolist()

dicionario_final['cidade_por_operadora'] = []

for operadora in dicionario_final['operadora']:
    frame_op = dados_brutos[dados_brutos['operadora'] == operadora]
    dic_op = {}
    dic_op['operadora'] = operadora
    dic_op['municipios'] = []
    for cidades in pd.unique(frame_op['cidade']):
        dic_op['municipios'].append(cidades)
    dicionario_final['cidade_por_operadora'].append(dic_op)

### dados agregados de estações, bicicletas, patinetes, viagens, distância
for col_errada in lista_colunas_trocar:
    dados_brutos[col_errada] = pd.to_numeric(dados_brutos[col_errada], errors='coerce')

lista_para_somar = ['estações','bicicletas_elétricas','bicicletas','patinetes_elétricos','viagens_diárias','média_distância_percorrida_por_dia','usuários','emissões_de_co2_evitadas']        

for col_para_somar in lista_para_somar:
    print(col_para_somar)
    col_para_somar
    if np.isnan(dados_brutos[col_para_somar].sum(min_count=1)) == True:
        dicionario_final[col_para_somar] = 'Dados não disponíveis'
        print(dados_brutos[col_para_somar])
    else:
        dicionario_final[col_para_somar] = dados_brutos[col_para_somar].sum(min_count=1)

## dados agregados de dados demográficos
lista_demograficos = ['mulheres','homens','15_até_29_anos','30_até_59_anos','acima_de_60']

for col_porcentagem in lista_demograficos:
    lista_valores_totais = []
    print(col_porcentagem)
    for municipio in pd.unique(dados_brutos['cidade']):
        frame_municipio = dados_brutos[dados_brutos['cidade'] ==  municipio]
        if np.isnan(frame_municipio['usuários'].sum(min_count=1)) == True:
            dicionario_final[col_porcentagem] = 'Dados não disponíveis'
        else:
            for sistema in pd.unique(frame_municipio['sistema']):
                frame_sistema = frame_municipio[frame_municipio['sistema'] == sistema]
                if np.isnan(frame_sistema['usuários'].sum(min_count=1)) == True:
                    dicionario_final[col_porcentagem] = 'Dados não disponíveis'
                else:
                    for veiculo in pd.unique(frame_sistema['tipologia_do_veículo']):
                        frame_final = frame_sistema[frame_sistema['tipologia_do_veículo'] == veiculo]
#                         if np.isnan(frame_final['usuários']) == True:
                        if frame_final['usuários'].empty == True:
                            dicionario_final[col_porcentagem] = 'Dados não disponíveis'
                        else:
                            valor_absoluto = (frame_final['usuários'].values*frame_final[col_porcentagem].values)/100
                            print('======== valor abs ========')
                            print(valor_absoluto[0])
                            print(valor_absoluto)
                            lista_valores_totais.append(valor_absoluto[0])                        
    if not lista_valores_totais:
        dicionario_final[col_porcentagem] = 'Dados não disponíveis'
    else:
        print(lista_valores_totais)
        dicionario_final[col_porcentagem] = functools.reduce((lambda x, y: x * y), lista_valores_totais)

Amazonas
Bahia
Ceará
Distrito Federal
Espírito Santo
Goiás
Minas Gerais
Pará
Paraná
Pernambuco
Rio de Janeiro
Rio Grande do Sul
Santa Catarina
São Paulo
estações
bicicletas_elétricas
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
48   NaN
49   NaN
50   NaN
51   NaN
52   NaN
53   NaN
Name: bicicletas_elétricas, dtype: float64
bicicletas
patinetes_elétricos
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [7]:
def cidadePorc(frame,dici):
    if frame[col_por].isna().values.any():
        dici[col_por] = 'Dados não disponíveis'
    else:
        filter_frame = frame[~pd.isnull(frame['usuários'])]
        null_frame = frame[pd.isnull(frame['usuários'])]
        
        if len(filter_frame.index) == 1:
            dici[col_por] = filter_frame[col_por].values[0]
        elif len(filter_frame.index) > 1:
            total_usuario = filter_frame['usuários'].sum()
            total_porc = filter_frame[col_por].sum()
            valor_absoluto = (total_porc/total_usuario)*100
            dici[col_por] = valor_absoluto
        else:
            if len(null_frame) == 1:
                dici[col_por] = null_frame[col_por].values[0]
            else:
                frame_sistema = frame[frame['sistema'] == sistema]

                if frame_sistema[col_por].isna().values.any():
                    dici[col_por] = 'Dados não disponíveis'
                else:
                    filter_frame = frame_sistema[~pd.isnull(frame['usuários'])]
                    null_frame = frame_sistema[pd.isnull(frame_sistema['usuários'])]

                    if len(filter_frame.index) == 1:
                        dici[col_por] = filter_frame[col_por].values[0]
                    elif len(filter_frame.index) > 1:
                        total_usuario = filter_frame['usuários'].sum()
                        total_porc = filter_frame[col_por].sum()
                        valor_absoluto = (total_porc/total_usuario)*100
                        dici[col_por] = valor_absoluto
                    else:
                        if len(null_frame.index) == 1:
                            check_columns(null_frame)
                            dici[col_por] = null_frame[col_por].values[0]
                        else:
                            total_usuario = null_frame['usuários'].sum()
                            total_porc = null_frame[col_por].sum()
                            valor_absoluto = (total_porc/total_usuario)*100
                            dici[col_por] = valor_absoluto
            
    return dici[col_por]

def getPorcentagens(frame,dici):
    
    if frame[col_por].isna().values.any():
        dici[col_por] = 'Dados não disponíveis'
    else:
        filter_frame = frame[~pd.isnull(frame['usuários'])]
        null_frame = frame[pd.isnull(frame['usuários'])]
        if len(filter_frame.index) == 1:
            dici[col_por] = filter_frame[col_por].values[0]
        elif len(filter_frame.index) > 1:
            total_usuario = filter_frame['usuários'].sum()
            total_porc = filter_frame[col_por].sum()
            valor_absoluto = (total_porc/total_usuario)*100
            dici[col_por] = valor_absoluto
        else:
            if len(null_frame.index) == 1:
                check_columns(null_frame)
                dici[col_por] = null_frame[col_por].values[0]
            else:
                total_usuario = null_frame['usuários'].sum()
                total_porc = null_frame[col_por].sum()
                valor_absoluto = (total_porc/total_usuario)*100
                dici[col_por] = valor_absoluto
            
    return dici[col_por]

In [8]:
dicionario_final['municipios'] = []

lista_para_somar_2 = ['estações','bicicletas_elétricas','bicicletas','patinetes_elétricos','viagens_diárias','média_distância_percorrida_por_dia','usuários','emissões_de_co2_evitadas']
lista_demograficos_2 = ['mulheres','homens','15_até_29_anos','30_até_59_anos','acima_de_60']


## ler todas as cidades no arquivo, e para cada cidade, criar um dicionario/objeto
## aos poucos, adicionar propriedades únicas como nome e outras coisas mais

for cidade_item in pd.unique(dados_brutos['cidade']):
    ## criar corte nos dados só com a cidade selecionada no momento dentro do loop
    ## vamos fazer calculos em cima disso
    frame_cidade = dados_brutos[dados_brutos['cidade'] == cidade_item]
    print(cidade_item)
    dic_cidade = {}
    dic_cidade['cidade'] = cidade_item.rstrip().lstrip()
    
    id_cidade_estado = cidade_item.rstrip().lstrip() + ' - ' + frame_cidade.iloc[0]['estado'].rstrip().lstrip()
#     dic_cidade['estado'] = id_cidade_estado
    print(id_cidade_estado)
    frame_com_latlong = dados_geo[dados_geo['cidade_estado'] == id_cidade_estado]
    print(str(len(frame_com_latlong)))
    dic_cidade['lat'] = frame_com_latlong.iloc[0]['latitude']
    dic_cidade['long'] = frame_com_latlong.iloc[0]['longitude']
    dic_cidade['estado'] = frame_cidade.iloc[0]['estado']
    dic_cidade['sistemas'] = []
    dic_cidade['modais'] = []
    
    ### dados gerais das cidades
    for col_para_somar in lista_para_somar_2:
        if np.isnan(frame_cidade[col_para_somar].sum(min_count=1)) == True:
            dic_cidade[col_para_somar] = 'Dados não disponíveis'
        else:
            dic_cidade[col_para_somar] = frame_cidade[col_para_somar].sum(min_count=1)
            
    ### dados demograficos das cidades
    for col_por in lista_demograficos_2:
        cidadePorc(frame_cidade,dic_cidade)
           
    ##########################################################################################
    ### dados agregados de sistemas
    for sistema in pd.unique(frame_cidade['sistema']):
        frame_sistema = frame_cidade[frame_cidade['sistema'] == sistema]
        dict_sistema = {}
        dict_sistema['sistema'] = sistema
        dict_sistema['cidade'] = dic_cidade['cidade']
#         dict_sistema['estado'] = dic_cidade['estado']        
        dict_sistema['lat'] = dic_cidade['lat']
        dict_sistema['long'] = dic_cidade['long']

        ### dados gerais dos sistemas
        for col_para_somar in lista_para_somar_2:
            if np.isnan(frame_sistema[col_para_somar].sum(min_count=1)) == True:
                dict_sistema[col_para_somar] = 'Dados não disponíveis'
            else:
                dict_sistema[col_para_somar] = frame_sistema[col_para_somar].sum(min_count=1)
        
        ### dados demograficos dos sistemas
        for col_por in lista_demograficos_2:
            getPorcentagens(frame_sistema,dict_sistema)
     
        ##########################################################################################
        ### Adicionar dicionario/objeto de cada veiculo dentro de cada sistema; passo final!
        dict_sistema['veiculos'] = []
        for veiculo in pd.unique(frame_sistema['tipologia_do_veículo']):
            print(veiculo)
            dict_veiculo = {}
            dict_veiculo['lat'] = dic_cidade['lat']
            dict_veiculo['long'] = dic_cidade['long']
            frame_veiculo = frame_sistema[frame_sistema['tipologia_do_veículo'] == veiculo]
            lista_colunas_veiculo = frame_veiculo.columns.values
            for colunas_linha in lista_colunas_veiculo:
                dict_veiculo[colunas_linha] = frame_veiculo.iloc[0][colunas_linha]
            dict_sistema['veiculos'].append(dict_veiculo)
        ##########################################################################################
        
        ### Adicionar sistemas e modais a sua cidade
        dic_cidade['sistemas'].append(dict_sistema)
    
    ##########################################################################################
    ### dados agregados de MODAIS
    for modal in pd.unique(frame_cidade['tipologia_do_veículo']):
        frame_modal = frame_cidade[frame_cidade['tipologia_do_veículo'] == modal]
        dict_modal = {}
        dict_modal['tipologia_do_veículo'] = modal
        dict_modal['cidade'] = dic_cidade['cidade']
#         dict_modal['estado'] = dic_cidade['estado']
        dict_modal['lat'] = dic_cidade['lat']
        dict_modal['long'] = dic_cidade['long']

        ### dados gerais dos modais
        for col_para_somar in lista_para_somar_2:
            if np.isnan(frame_modal[col_para_somar].sum(min_count=1)) == True:
                dict_modal[col_para_somar] = 'Dados não disponíveis'
            else:
                dict_modal[col_para_somar] = frame_modal[col_para_somar].sum(min_count=1)
        
        ### dados demograficos dos sistemas
        for col_por in lista_demograficos_2:
            getPorcentagens(frame_modal,dict_modal)
        
        dic_cidade['modais'].append(dict_modal)
        
    ##########################################################################################
    ### Adicionar cidades ao objeto final
    dic_cidade
    dicionario_final['municipios'].append(dic_cidade)

Belém
Belém - Pará
1
Bicicleta
Belo Horizonte
Belo Horizonte - Minas Gerais
1
Bicicleta
Bicicleta
Patinete Eletrico
Bertioga
Bertioga - São Paulo
1
Bicicleta
Brasília
Brasília - Distrito Federal
1
Bicicleta
Patinete Eletrico
Campinas
Campinas - São Paulo
1
Bicicleta
Patinete Eletrico
Curitiba
Curitiba - Paraná
1
Bicicleta
Patinete Eletrico
Florianópolis
Florianópolis - Santa Catarina
1
Bicicleta
Patinete Eletrico
Fortaleza
Fortaleza - Ceará
1
Bicicleta
Bicicleta
Goiânia
Goiânia - Goiás
1
Bicicleta
Ilhabela
Ilhabela - São Paulo
1
Bicicleta
Juiz de Fora
Juiz de Fora - Minas Gerais
1
Bicicleta
Manaus
Manaus - Amazonas
1
Bicicleta
Maricá
Maricá - Rio de Janeiro
1
Bicicleta
Pelotas
Pelotas - Rio Grande do Sul
1
Bicicleta
Petrolina
Petrolina - Pernambuco
1
Bicicleta
Porto Alegre
Porto Alegre - Rio Grande do Sul
1
cidade:  1
['Porto Alegre']
++++++++++++++++++
estado:  1
['Rio Grande do Sul']
++++++++++++++++++
região:  1
['Sul']
++++++++++++++++++
sistema:  1
['Bike PoA']
++++++++++++++++++


++++++++++++++++++
estado:  1
['Pernambuco']
++++++++++++++++++
região:  1
['Nordeste']
++++++++++++++++++
sistema:  1
['Bike PE']
++++++++++++++++++
operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[80.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[800.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[2821.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[14044.01]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[26.38]
++++++++++++++++++
homens:  1
[65.87]
++++++++++++++++++
15_até_29_anos:  1
[58.66]
++++++++++++++++++
30_até_59_anos:  1
[36.31]
++++++++++++++++++
acima_de_60:  1
[1.38]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[74.34]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Metodologia do cálculo desenvolvida em 

tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[50.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[400.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[1284.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[8263.34]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[28.55]
++++++++++++++++++
homens:  1
[65.83]
++++++++++++++++++
15_até_29_anos:  1
[40.61]
++++++++++++++++++
30_até_59_anos:  1
[51.13]
++++++++++++++++++
acima_de_60:  1
[3.95]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[38.3]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Metodologia do cálculo desenvolvida em colaboração com o Instituto de Energia e Meio Ambiente (IEMA). \nNº de estações, nº de veículos, viagens diárias, distância percorrida por dia, sexo e idade: Calculados com base nos dados fornecido

In [9]:
dicionario_final

{'15_até_29_anos': 8.932864539204003e+33,
 '30_até_59_anos': 4.87806267592609e+34,
 'acima_de_60': 0.0,
 'bicicletas': 10520.0,
 'bicicletas_elétricas': 'Dados não disponíveis',
 'cidade_por_operadora': [{'municipios': ['Belém',
    'Bertioga',
    'Juiz de Fora',
    'Manaus',
    'Porto Alegre',
    'Recife',
    'Rio de Janeiro',
    'Salvador',
    'Santos',
    'São Paulo',
    'Sorocaba',
    'Vila Velha'],
   'operadora': 'Tembici'},
  {'municipios': ['Belo Horizonte',
    'Brasília',
    'Fortaleza',
    'Goiânia',
    'Maricá',
    'Pelotas',
    'Petrolina',
    'Santos',
    'São José dos Campos',
    'Serra',
    'Vitória'],
   'operadora': 'Serttel'},
  {'municipios': ['Belo Horizonte',
    'Brasília',
    'Campinas',
    'Curitiba',
    'Florianópolis',
    'Ilhabela',
    'Porto Alegre',
    'Rio de Janeiro',
    'São José dos Campos',
    'São Paulo',
    'Vila Velha',
    'Vitória'],
   'operadora': 'Yellow'},
  {'municipios': ['Rio de Janeiro', 'São Paulo'], 'operador

In [10]:
with open('teste_operadora.json', 'w') as json_file:
  json.dump(dicionario_final, json_file, encoding='utf-16',ignore_nan=True,ensure_ascii =False)